In [190]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from lxml import etree
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer

[nltk_data] Downloading package punkt to /home/ebryx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ebryx/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [191]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [192]:
from sklearn.preprocessing import MultiLabelBinarizer

In [193]:
from sklearn.model_selection import train_test_split

In [194]:
t1=time.time()
path='./dataset/blogs_train/'
all_files = glob.glob(path + "/*.xml")
li = []

for filename in all_files:
  # print("file name ", filename)
  li.append(filename)
    

t2=time.time()
print("time taken to read All csv :: ", t2-t1, " seconds ")

time taken to read All csv ::  0.03737950325012207  seconds 


In [195]:
def clean_file_names(li):
  # names_list=[]
  # for i in range(len(li)):
  name_list=li.split('/')[-1]
  return name_list

In [196]:


def get_attributes_from_name(name):
  # data_object=[]
  # for data in range(len(names_list)):
  #   dummy_var=names_list[data].split('.')
  #   dummy_object={'id':dummy_var[0], 'gender':dummy_var[1],
  #                 'age':dummy_var[2], 'category':dummy_var[3],
  #                 'horoscope':dummy_var[4]
  #                 }
  #   data_object.append(dummy_object)
  dummy_var=name.split('.')
  data_object={'id':dummy_var[0], 'gender':dummy_var[1],
                  'age':dummy_var[2], 'category':dummy_var[3],
                  'horoscope':dummy_var[4]
                }
  
  return data_object



In [197]:
parser = etree.XMLParser(encoding='ISO-8859-1', recover = True, huge_tree=True)

In [198]:
def read_xml(li):
    
    
    all_objects=[]
    for i in range(len(li)):
    # print("i will run ", i+1," times")
        name_of_file=clean_file_names(li[i])
    # print(name_of_file)
        data_object=get_attributes_from_name(name_of_file)
    # print(data_object)

        post_text='  '
        tree = etree.parse(li[i], parser=parser)
        root = tree.getroot()
        for element in root.iter():
            if element.tag == 'post':
                element_text = element.text
                post_text = post_text + element_text
        mystring = post_text.replace('\n', '').replace('\r', '').replace('\t', '')
        mystring = mystring.replace('   ', '').replace('\\','')
   


   
    # print(mystring)
    # name_of_file=clean_file_names(li[i])
    # data_object=get_attributes_from_name(name_of_file)
    



        data_object['posts']=mystring   
        all_objects.append(data_object)

    return all_objects

In [199]:
t1=time.time()
data=read_xml(li[0:])
t2=time.time()

In [200]:
stop_words = set(stopwords.words('english')) 
def remove_stop_words(posts):
    filtered_post=[]
    
    word_tokens = word_tokenize(posts) 
    filtered_sentence = []   
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)

    filtered_post.append(filtered_sentence)
    return filtered_post

In [201]:
stop_words = set(stopwords.words('english')) 
def fun_remove_stop_words(posts):
    filtered=''
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    return filtered    

In [202]:
for i in range(len(data)):
    data[i]['cleaned_stop_words']=fun_remove_stop_words(data[i]['posts'])

In [203]:
def remove_dashes(data):
    for i in range(len(data)):
        data[i]['cleaned_stop_words']=data[i]['cleaned_stop_words'].replace('--', ' ')
        

In [204]:
remove_dashes(data)

In [205]:
df=pd.DataFrame(data)

In [206]:
X=df[['age','cleaned_stop_words']]

In [207]:
dummy_variable_1 = pd.get_dummies(df["category"])
X=pd.concat([X,dummy_variable_1], axis=1)

In [208]:
del dummy_variable_1

In [209]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [210]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/ebryx/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [211]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(X['cleaned_stop_words']).toarray()

In [212]:
# vectorizer = CountVectorizer()
# vectorizer.fit(X['cleaned_stop_words'])
# posts = vectorizer.transform(X['cleaned_stop_words'])


In [213]:
posts

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 3, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 4, 0, 2]])

In [214]:
# transformed_posts = np.array([row.toarray() for row in posts])

In [215]:
# transformed_posts.shape=(100,35680)

In [216]:
transformed_posts=pd.DataFrame(posts)

In [217]:
X=pd.concat([X,transformed_posts],axis=1)

In [218]:
del transformed_posts

In [219]:
X=X.drop(['cleaned_stop_words'], axis=1)

In [220]:
X

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,15,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,25,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,24,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,17,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14595,15,0,0,0,0,0,0,0,0,0,...,0,0,9,17,4,25,0,14,2,3
14596,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
14597,16,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
14598,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [221]:
X.groupby('age').first().index

Index(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46',
       '47', '48'],
      dtype='object', name='age')

In [222]:
X['age']=X['age'].astype(int)

In [223]:
from scipy import stats

In [224]:
t1=time.time()

X['age'] = stats.zscore(X['age'])
    
t2=time.time()
print("time ::: ", t2-t1 , " seconds to find z score")

time :::  0.0012717247009277344  seconds to find z score


In [225]:
X.head()

,age,Accounting,Advertising,Agriculture,Architecture,Arts,Automotive,Banking,Biotech,BusinessServices,...,990,991,992,993,994,995,996,997,998,999
0,-1.002358,0,0,0,0,0,0,0,0,0,...,1,1,13,4,1,1,3,1,0,0
1,0.252330,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,2,3,0,0
3,0.126861,0,0,0,0,0,0,0,0,0,...,2,0,37,15,32,18,8,19,3,9
4,-0.751421,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,10,1,1,0


In [226]:
labels= []
for hor,gen in zip(df['horoscope'],df['gender']):
    labels.append((hor,gen))

In [227]:
mlb = MultiLabelBinarizer()
mlb.fit(labels)
classes=len(mlb.classes_)
print("classes  :: ", classes)
mlb.classes_

classes  ::  14


array(['Aquarius', 'Aries', 'Cancer', 'Capricorn', 'Gemini', 'Leo',
       'Libra', 'Pisces', 'Sagittarius', 'Scorpio', 'Taurus', 'Virgo',
       'female', 'male'], dtype=object)

In [228]:
labels = mlb.fit_transform(labels)

In [229]:
y=labels

In [230]:
trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.2, random_state=42)

### NN

In [231]:
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

In [232]:


input_dim = trainX.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(20, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(10,  activation='tanh'))
# model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))

model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(classes, activation='softmax'))
# model.add(Activation('softmax'))



In [233]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 20)                20840     
_________________________________________________________________
dense_10 (Dense)             (None, 10)                210       
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              11264     
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 14)                14350     
Total params: 50,760
Trainable params: 48,712
Non-trainable params: 2,048
_________________________________________________________________


In [234]:
trainX, testX, trainY, testY
classifier_nn = model.fit(trainX,trainY,
                    epochs=5,
                    verbose=True,
                    validation_data=(testX, testY),
                    batch_size=10)



Train on 11680 samples, validate on 2920 samples
Epoch 1/5
11680/11680 [==============================] - 6s 500us/step - loss: 0.3894 - acc: 0.8570 - val_loss: 0.3986 - val_acc: 0.8571
Epoch 2/5
11680/11680 [==============================] - 4s 376us/step - loss: 0.3801 - acc: 0.8576 - val_loss: 0.4653 - val_acc: 0.8573
Epoch 3/5
11680/11680 [==============================] - 4s 374us/step - loss: 0.3785 - acc: 0.8592 - val_loss: 0.4448 - val_acc: 0.8603
Epoch 4/5
11680/11680 [==============================] - 4s 373us/step - loss: 0.3754 - acc: 0.8619 - val_loss: 0.4286 - val_acc: 0.8666
Epoch 5/5
11680/11680 [==============================] - 4s 373us/step - loss: 0.3718 - acc: 0.8648 - val_loss: 0.4087 - val_acc: 0.8639


In [235]:
testX.shape

(2920, 1041)

In [236]:
testX[2].shape

(2920,)

In [237]:
prediction=model.predict(testX)

In [238]:
for i in prediction:
    print(i)
   

[0.06490546 0.047088   0.01118565 0.01556532 0.00400149 0.15477815
 0.11165486 0.03407676 0.02500933 0.0507097  0.02722022 0.01158568
 0.33228743 0.10993192]
[0.06419011 0.05262938 0.01223184 0.01569306 0.00444316 0.16148403
 0.10932959 0.03447539 0.02507416 0.05713873 0.02938533 0.01112772
 0.28520733 0.13759024]
[0.06442644 0.05279844 0.01228368 0.01571697 0.00445521 0.1614581
 0.10922307 0.03452555 0.0251378  0.05735377 0.02943485 0.01119327
 0.2835064  0.13848644]
[0.03972966 0.02258732 0.00700483 0.0113747  0.00204532 0.14190577
 0.04205861 0.02414996 0.01388606 0.02900462 0.0176113  0.00972705
 0.6232556  0.01565916]
[0.06468326 0.05303335 0.01234206 0.01573859 0.00447197 0.16132282
 0.10927353 0.03457373 0.02521586 0.05761629 0.02949455 0.01125314
 0.280934   0.14004685]
[0.0447512  0.0237152  0.00786603 0.01221821 0.00212342 0.14649159
 0.03796519 0.02569337 0.01468599 0.03135996 0.0183593  0.01246891
 0.60703516 0.01526648]
[0.06417608 0.04653709 0.01204086 0.01592932 0.003985

[0.07443614 0.05902939 0.01450549 0.01660484 0.00488481 0.15942435
 0.10119504 0.03632477 0.02751851 0.06606413 0.0311202  0.0144134
 0.22335336 0.1711256 ]
[0.07280926 0.05947089 0.0142174  0.016278   0.00491184 0.15577078
 0.10556871 0.03571489 0.02733372 0.06551774 0.03089943 0.01355699
 0.21199459 0.18595573]
[0.08543554 0.0671274  0.01725906 0.01670007 0.00533776 0.14218014
 0.08844755 0.03607864 0.02954194 0.07714303 0.03136864 0.01869067
 0.13357878 0.25111082]
[0.05697841 0.03769928 0.01046725 0.01481797 0.00329073 0.16645929
 0.0695339  0.03188934 0.02057598 0.04537445 0.02495528 0.01238202
 0.45729572 0.04828023]
[0.08409534 0.0680278  0.01735288 0.01535756 0.00534678 0.12470179
 0.08448906 0.03359196 0.02884016 0.07751392 0.03004811 0.01843496
 0.09420496 0.3179947 ]
[0.07593966 0.04819309 0.01414911 0.01774797 0.00402608 0.17624432
 0.06713057 0.03696414 0.02520343 0.05975271 0.02878582 0.01929407
 0.35634232 0.07022678]
[0.05421646 0.03670553 0.00992348 0.01440701 0.003215

 0.09343627 0.29439   ]
[0.05191305 0.03409012 0.00944294 0.01396895 0.00300304 0.16160981
 0.0661106  0.03005211 0.01895075 0.04100013 0.02331336 0.01110539
 0.49566126 0.03977841]
[0.04146651 0.02348155 0.0073364  0.01171048 0.00211899 0.1445327
 0.04247743 0.02484286 0.01436503 0.03026417 0.01815294 0.0103196
 0.612363   0.01656846]
[0.06197783 0.04709883 0.01383004 0.00717896 0.00355126 0.03024127
 0.02205053 0.01313128 0.01765598 0.06011542 0.01314894 0.01809684
 0.00481073 0.687112  ]
[0.06416657 0.05261011 0.01222659 0.01569084 0.00444178 0.1614912
 0.10933191 0.0344706  0.02506737 0.05711597 0.02938006 0.01112176
 0.2854111  0.13747415]
[0.06459597 0.05039097 0.01223381 0.01588818 0.00427848 0.16573273
 0.10038383 0.03473516 0.02469071 0.05610161 0.02906961 0.01187452
 0.3164261  0.11359829]
[0.06661417 0.05461469 0.01277768 0.01590271 0.00458355 0.16048154
 0.10891431 0.03492746 0.0257596  0.0594954  0.0298991  0.01175529
 0.26413086 0.15014364]
[0.06575889 0.05391442 0.012583

 0.03425751 0.47291523]
[0.05371181 0.03442543 0.00656429 0.01803109 0.00364797 0.18022032
 0.05612762 0.03085573 0.02425093 0.04281555 0.01095466 0.01139537
 0.48767245 0.03932679]
[0.05241846 0.03067245 0.00941614 0.01380254 0.00271104 0.15890138
 0.05197196 0.02932775 0.01789907 0.03903839 0.02197035 0.0130275
 0.5310484  0.02779462]
[0.07626254 0.06234879 0.01508221 0.01629027 0.00509107 0.15014705
 0.10245652 0.03572861 0.02809525 0.06909982 0.03117788 0.01467388
 0.17829762 0.21524854]
[0.09342869 0.06006235 0.01651243 0.01994257 0.00519299 0.14389952
 0.07553301 0.03596963 0.02859655 0.08246797 0.03450668 0.01557286
 0.27891403 0.10940067]
[0.06871111 0.04424522 0.01242253 0.0169671  0.00367332 0.16869873
 0.07175141 0.035109   0.02373791 0.05260676 0.02692013 0.01605838
 0.39608753 0.06301098]
[0.06416577 0.05260944 0.01222641 0.01569077 0.00444173 0.16149142
 0.10933197 0.03447044 0.02506714 0.05711517 0.02937988 0.01112155
 0.2854182  0.13747014]
[0.06416578 0.05260944 0.0122

 0.23988679 0.14630954]
[0.04377295 0.02594136 0.0078013  0.01226785 0.00232845 0.14938335
 0.04842759 0.02613819 0.01546351 0.03272838 0.01942589 0.01034801
 0.5849655  0.02100771]
[0.07361726 0.05810619 0.01625458 0.00984139 0.00437306 0.04900537
 0.03263872 0.01846896 0.02147515 0.07389612 0.01730315 0.01998206
 0.01170158 0.59333646]
[0.06416578 0.05260944 0.01222641 0.01569077 0.00444173 0.16149141
 0.10933197 0.03447043 0.02506714 0.05711518 0.02937987 0.01112155
 0.28541818 0.13747014]
[0.05263315 0.03478815 0.00959    0.01410295 0.00305996 0.16242512
 0.06744512 0.03035976 0.01924253 0.04172377 0.02362018 0.01120112
 0.48811677 0.04169135]
[0.0592117  0.04307135 0.01100255 0.01525092 0.00372103 0.16723081
 0.08628046 0.03312801 0.02230491 0.04941334 0.02683676 0.01149304
 0.39823717 0.07281802]
[0.06523354 0.04432008 0.01199106 0.01618819 0.00384672 0.17341407
 0.07558394 0.0345998  0.02348143 0.05328417 0.0267954  0.01433634
 0.38875353 0.06817174]
[0.03425088 0.01778316 0.005

In [239]:
testX.shape

(2920, 1041)

In [240]:
testX.iloc[1].shape

(1041,)

In [241]:
testX.iloc[1].dtypes

dtype('float64')

In [242]:
testX.iloc[1]

age            -1.002358
Accounting      0.000000
Advertising     0.000000
Agriculture     0.000000
Architecture    0.000000
                  ...   
995             6.000000
996             1.000000
997             1.000000
998             0.000000
999             0.000000
Name: 9354, Length: 1041, dtype: float64

In [243]:
a=np.zeros((1,1041))

In [244]:
sample_test=testX.iloc[1]

In [245]:
a[0][1]

0.0

In [246]:
for i in range(len(sample_test)):
    a[0][i]=sample_test.iloc[i]

In [247]:
a

array([[-1.00235836,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ]])

In [248]:
model.predict(a)

array([[0.06419008, 0.05262936, 0.01223183, 0.01569305, 0.00444315,
        0.16148403, 0.10932954, 0.03447536, 0.02507415, 0.0571387 ,
        0.02938532, 0.01112771, 0.2852075 , 0.13759014]], dtype=float32)

In [249]:
testY[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

In [250]:
for i in model.predict(testX):
    print(i)

[0.06490546 0.047088   0.01118565 0.01556532 0.00400149 0.15477815
 0.11165486 0.03407676 0.02500933 0.0507097  0.02722022 0.01158568
 0.33228743 0.10993192]
[0.06419011 0.05262938 0.01223184 0.01569306 0.00444316 0.16148403
 0.10932959 0.03447539 0.02507416 0.05713873 0.02938533 0.01112772
 0.28520733 0.13759024]
[0.06442644 0.05279844 0.01228368 0.01571697 0.00445521 0.1614581
 0.10922307 0.03452555 0.0251378  0.05735377 0.02943485 0.01119327
 0.2835064  0.13848644]
[0.03972966 0.02258732 0.00700483 0.0113747  0.00204532 0.14190577
 0.04205861 0.02414996 0.01388606 0.02900462 0.0176113  0.00972705
 0.6232556  0.01565916]
[0.06468326 0.05303335 0.01234206 0.01573859 0.00447197 0.16132282
 0.10927353 0.03457373 0.02521586 0.05761629 0.02949455 0.01125314
 0.280934   0.14004685]
[0.0447512  0.0237152  0.00786603 0.01221821 0.00212342 0.14649159
 0.03796519 0.02569337 0.01468599 0.03135996 0.0183593  0.01246891
 0.60703516 0.01526648]
[0.06417608 0.04653709 0.01204086 0.01592932 0.003985

 0.46330118 0.04877067]
[0.06415575 0.0525696  0.012223   0.01569222 0.00443886 0.16155623
 0.1092267  0.03447166 0.0250585  0.05708789 0.02937348 0.01112624
 0.28595048 0.13706933]
[0.08023889 0.06655876 0.01634645 0.01527616 0.00611003 0.11683702
 0.06424962 0.03111606 0.0298219  0.08147647 0.0221687  0.01933462
 0.05700082 0.39346448]
[0.05189572 0.0141396  0.00431838 0.01022395 0.00118807 0.07198714
 0.09041077 0.02286232 0.01454209 0.01362331 0.01094292 0.01376211
 0.66528136 0.01482231]
[0.06416578 0.05260944 0.01222641 0.01569077 0.00444173 0.16149141
 0.10933197 0.03447043 0.02506714 0.05711518 0.02937987 0.01112155
 0.28541818 0.13747014]
[0.06800868 0.05312948 0.01518295 0.00812052 0.00401012 0.03793618
 0.02806314 0.01557782 0.01898881 0.06930386 0.01591773 0.0174887
 0.0078583  0.6404137 ]
[0.06802409 0.05577349 0.01310522 0.01601595 0.00466614 0.15966153
 0.10847482 0.03515727 0.02615238 0.06088935 0.0301708  0.01213465
 0.2516588  0.1581155 ]
[0.06280998 0.04901742 0.0118

 0.5923427  0.01993151]
[0.06885941 0.04219715 0.01263558 0.01669872 0.00364437 0.17623036
 0.06060183 0.03504092 0.02308708 0.05371363 0.02620958 0.01776411
 0.41180474 0.05151244]
[0.06565766 0.05383151 0.01256105 0.0158243  0.00452853 0.16094105
 0.10912111 0.03475866 0.02549254 0.05856265 0.02970346 0.01150424
 0.27246925 0.145044  ]
[0.05147151 0.03362631 0.00935217 0.01388349 0.00296531 0.16108644
 0.06512267 0.02985343 0.0187621  0.04053978 0.02310743 0.01106332
 0.5006733  0.03849276]
[0.06416578 0.05260944 0.01222641 0.01569077 0.00444173 0.16149141
 0.10933197 0.03447043 0.02506714 0.05711518 0.02937987 0.01112155
 0.28541818 0.13747014]
[0.07813632 0.06303606 0.0154915  0.01646003 0.00513534 0.15030149
 0.09962199 0.03603711 0.02842755 0.07050567 0.0313592  0.01545924
 0.17384419 0.2161843 ]
[0.06605574 0.05415754 0.01265101 0.01585764 0.0045515  0.16076018
 0.10904218 0.03483047 0.02560425 0.05895042 0.02978605 0.01160816
 0.26900223 0.14714254]
[0.11563136 0.07506279 0.020

 0.4660903  0.04451511]
[0.06587847 0.04648345 0.01237469 0.01615555 0.00397737 0.17107014
 0.08290578 0.0349701  0.02394722 0.05456151 0.02828199 0.01378641
 0.3653448  0.08026242]
[0.06416546 0.05260834 0.0122263  0.01569081 0.00444165 0.16149321
 0.10932905 0.03447046 0.02506686 0.05711441 0.02937971 0.01112167
 0.28543293 0.13745911]
[0.05766899 0.04050149 0.01031702 0.01475257 0.00380531 0.17563315
 0.07939089 0.03309657 0.02203667 0.04844354 0.0243951  0.01180463
 0.41372356 0.06443046]
[0.04883252 0.02727907 0.00844495 0.0130277  0.00246533 0.15340713
 0.0466693  0.02756069 0.01672085 0.0351188  0.01998039 0.01247167
 0.566181   0.02184059]
[0.06416541 0.05260808 0.01222629 0.01569081 0.00444163 0.16149358
 0.10932846 0.03447047 0.02506683 0.05711426 0.02937965 0.0111217
 0.28543624 0.13745666]
[0.06525672 0.05348668 0.01247009 0.01579112 0.00450415 0.1611429
 0.10913835 0.03468614 0.02537638 0.05816355 0.02961674 0.01140408
 0.27618867 0.14277445]
[0.06416578 0.05260944 0.01222

In [251]:
for j in testY:
    print(j)

[1 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 1 0 0 0 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 1 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1 1 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0

[1 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 1 0 0 0 0 1 0]
[0 0 0 0 0 1 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 1 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 1 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[1 0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1 0]
[0 1 0 0 0 0 0 0 0 0 0 0 0 1]
[0 1 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 1 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 1 0 0 0 0 0 0 0 1 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0

In [252]:
sample_test=testX.iloc[100]

In [253]:
for i in range(len(sample_test)):
    a[0][i]=sample_test.iloc[i]

In [254]:
model.predict(a)

array([[0.07986583, 0.06373521, 0.01586889, 0.01655503, 0.0051944 ,
        0.14997295, 0.09704115, 0.03626394, 0.02875593, 0.07191287,
        0.03143014, 0.01620697, 0.1674038 , 0.21979296]], dtype=float32)

In [255]:
testY[100]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

### End of NN

## KNN

In [50]:
from sklearn.neighbors import KNeighborsClassifier

In [51]:
neigh = KNeighborsClassifier(n_neighbors=4)

In [52]:
neigh.fit(trainX,trainY)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [55]:
neigh.predict_proba(testX)

[array([[1.  , 0.  ],
        [0.75, 0.25],
        [1.  , 0.  ],
        ...,
        [1.  , 0.  ],
        [1.  , 0.  ],
        [1.  , 0.  ]]),
 array([[0.75, 0.25],
        [1.  , 0.  ],
        [1.  , 0.  ],
        ...,
        [1.  , 0.  ],
        [0.75, 0.25],
        [1.  , 0.  ]]),
 array([[1.  , 0.  ],
        [1.  , 0.  ],
        [1.  , 0.  ],
        ...,
        [1.  , 0.  ],
        [1.  , 0.  ],
        [0.75, 0.25]]),
 array([[0.75, 0.25],
        [1.  , 0.  ],
        [1.  , 0.  ],
        ...,
        [0.5 , 0.5 ],
        [0.75, 0.25],
        [0.5 , 0.5 ]]),
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]),
 array([[1.  , 0.  ],
        [1.  , 0.  ],
        [0.75, 0.25],
        ...,
        [1.  , 0.  ],
        [0.75, 0.25],
        [1.  , 0.  ]]),
 array([[0.75, 0.25],
        [1.  , 0.  ],
        [0.75, 0.25],
        ...,
        [0.75, 0.25],
        [0.75, 0.25],
        [1.  , 0.  

In [117]:
neigh.predict(a)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [118]:
testY[100]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1])

### End of KNN

### Word2 Vector implementation

In [ ]:
words_of_posts=[]
for j in range(len(data)):
    posts=data[j]['posts']
    
    words=[]
    for i in range(len(posts)):
        words.append(word_tokenize(posts[i]))
    words_of_posts.append(words)

In [ ]:
remove_stop_words(data[1]['posts'])

In [ ]:
if len(data)==len(words_of_posts):
    for i in range(len(data)):
        data[i]['tokenized']=words_of_posts[i]

In [ ]:
df=pd.DataFrame(data)

In [ ]:
X=df['tokenized'].values

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
y=data['horoscope']

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

In [ ]:
y_transform=le.transform(y)

In [ ]:
from keras.utils import np_utils

In [ ]:
dummy_y = np_utils.to_categorical(y_transform)

In [ ]:
#Split

In [ ]:
# model

In [ ]:
# compile

In [ ]:
# fit

In [ ]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [ ]:
freq_matrix=_create_frequency_matrix(data[1]['posts'])

In [ ]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [ ]:
tf_matrix=_create_tf_matrix(freq_matrix)

In [ ]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [ ]:
doc_per_word=_create_documents_per_words(freq_matrix)

In [ ]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [ ]:
_create_idf_matrix(freq_matrix,doc_per_word,total_documents)